In [2]:
import ee
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [ ]:
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              8.473892211914062,
              49.98081240937428
            ],
            [
              8.658599853515625,
              49.98081240937428
            ],
            [
              8.658599853515625,
              50.06066538593667
            ],
            [
              8.473892211914062,
              50.06066538593667
            ],
            [
              8.473892211914062,
              49.98081240937428
            ]
          ]
        ]
      }
    }
  ]
}

In [ ]:
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

In [ ]:
27.129564922715716,54.55896956167419,
27.14295451011806,54.55896956167419,
27.14295451011806,54.566932014707845,
27.129564922715716,54.566932014707845,
27.129564922715716,54.55896956167419




26.992805034442117,54.60416673414815,
27.011086971087625,54.60416673414815,
27.011086971087625,54.61714028506184,
26.992805034442117,54.61714028506184,
26.992805034442117,54.60416673414815

In [ ]:
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

In [28]:
import pandas as pd
from dateutil.relativedelta import relativedelta
# Any region of the world
polygon = ee.Geometry.Polygon([26.992805034442117,54.60416673414815,
27.011086971087625,54.60416673414815,
27.011086971087625,54.61714028506184,
26.992805034442117,54.61714028506184,
26.992805034442117,54.60416673414815
])
startDate = '2022-04-01'
endDate = '2022-11-30'

modisNDVI = ee.ImageCollection('MODIS/MOD09GA_006_NDVI').select('NDVI').filterDate(startDate, endDate)
# Select the bands for NDVI calculation


start_date = pd.to_datetime(startDate)
end_date = pd.to_datetime(endDate)

# Calculate the number of months between the two dates
num_months = relativedelta(end_date, start_date).months + (relativedelta(end_date, start_date).years * 12)

print(num_months)
    
def custom_fun(n):
    date = ee.Date(startDate).advance(n,'month')
    m = date.get("month")
    y = date.get("year")
    dic = ee.Dictionary({
        'Date':date.format('yyyy-MM')
    })
    
    tempNDVI = (modisNDVI.filter(ee.Filter.calendarRange(y, y, 'year'))
                .filter(ee.Filter.calendarRange(m, m, 'month'))
                .mean()
                .reduceRegion(
                    reducer = ee.Reducer.mean(),
                    geometry = polygon,
                    scale = 250))

    return dic.combine(tempNDVI)

modis_YrMo = ee.List.sequence(0, num_months).map(custom_fun)

dataframe = pd.DataFrame(modis_YrMo.getInfo())

7


In [29]:
dataframe.head(num_months)

,Date,NDVI
0,2022-04,0.260003
1,2022-05,0.295145
2,2022-06,0.480019
3,2022-07,0.343241
4,2022-08,0.558420
5,2022-09,0.245553
6,2022-10,0.164912


In [25]:
dataframe['Date'] = pd.to_datetime(dataframe['Date'])

dataframe = dataframe[(dataframe['Date'].dt.month >= 3) & (dataframe['Date'].dt.month <= 6) | (dataframe['Date'].dt.month >= 9) & (dataframe['Date'].dt.month <= 11)]

march_to_may = (dataframe['Date'].dt.month >= 3) & (dataframe['Date'].dt.month <= 5)
sept_to_nov = (dataframe['Date'].dt.month >= 9) & (dataframe['Date'].dt.month <= 11)

if (dataframe[march_to_may]['NDVI'].min() < 0.16) & (dataframe[sept_to_nov]['NDVI'].min() < 0.16):
    print("Changes detected")
else:
    print("Changes not detected")

Changes not detected
